# Inicio

In [1]:
print("hello world")

hello world


In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.src.legacy.preprocessing.image import ImageDataGenerator
import os
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score

2024-10-13 19:35:22.365265: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-13 19:35:22.670630: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2024-10-13 19:35:23.254140: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-13 19:35:23.272230: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-13 19:35:23.275439: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

## Definindo diretório de imagens
Definindo o diretório que contém as pastas contendo as imagens categorizadas.

In [4]:
# Mapear arquivos ocultos (.ipny)
# A ser utilizada caso não tenha o .ipynb_checkpoints
def filter_hidden_folders(folder_list):
    return [folder for folder in folder_list if not folder.startswith('.')]

In [5]:
# Caminho para os dados do dataset e o reescalador de imagens
xray_directory = 'font_results/cropped'

xray_classes = os.listdir(xray_directory)
print(f"Classes: {xray_classes}")

class_count = len(os.listdir(xray_directory))
print(f"Quantidade de classes:{class_count} => 26 letras maiúsculas + 26 minúsculas + 1 nula")

Classes: ['g', 'V', 'W', 'r', 'i', 'a', 'G', 'I', 's', 'J', 'y', 'z', 't', 'c', 'Q', 'u', 'E', 'o', 'H', 'B', 'U', 'N', 'F', 'm', 'n', 'v', 'T', 'D', 'A', 'e', 'w', 'O', '.ipynb_checkpoints', 'S', 'K', 'l', 'C', 'q', 'p', 'Z', 'j', 'h', 'f', 'L', 'M', 'b', 'd', 'P', 'x', 'X', 'Y', 'R', 'k']
Quantidade de classes:53 => 26 letras maiúsculas + 26 minúsculas + 1 nula


In [6]:
# Definir parâmetros
INPUT_SHAPE = (120, 100, 1)  # Imagem 120x100, 1 canal (grayscale)

In [7]:
# Definindo o modelo
model = Sequential()


# Primeira camada de convolução e pooling
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=INPUT_SHAPE))
model.add(layers.MaxPooling2D((2, 2)))

# Segunda camada de convolução e pooling
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Terceira camada de convolução e pooling
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Quarta camada de convolução e pooling
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Camada de flatten
model.add(layers.Flatten())

# Camadas densas (fully connected)
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.5))  # Evitar overfitting

# Camada de saída
model.add(layers.Dense(class_count, activation='softmax'))

/home/picg/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-10-13 19:35:23.288615: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-13 19:35:23.291804: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing

In [8]:
# Compilando o modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Resumo do modelo
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 118, 98, 32)    │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 59, 49, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 57, 47, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 28, 23, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 26, 21, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 13, 10, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 11, 8, 128)     │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 5, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2560)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     1,311,232 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 53)             │        27,189 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,578,677 (6.02 MB)

 Trainable params: 1,578,677 (6.02 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
# Salvar o arquivo de treinamento em diferentes epocas

# Nome/caminho do arquivo que sera salvo
filepath = "weights_cropped_original.keras" 
#filepath = "weights_normal_original.keras" 

# Define os parametros para atualizar o arquivo final
# ModelCheckpoint(nome_do_arquivo, monitor = 'parametro a ser alalisado', verbose = 1 (imprimir no console quando arquivo for salvo), save_best_only = True, mode = 'min' (objetivo minimizar perda))
checkpoint = ModelCheckpoint(filepath, monitor = 'loss', verbose = 1, save_best_only = True, mode = 'min')

# Cria uma lista que contém o ModelCheckpoint configurado usado durante o treinamento
callbacks_list = [checkpoint]

In [10]:
# Gerenciador de imagem para pre processamento
# Aumentar a base de dados 

train_datagen = ImageDataGenerator(
# Transforma os valores de pixel de (0-255 ->0.0 - 1.0 )  
    rescale=1. / 255,
# Aplica zoom aleatório às imagens dentro do intervalo especificado (0.2 -> 20%)
    zoom_range=0.2,
# Realiza uma inversão horizontal aleatória das imagens
    horizontal_flip=True,
#Este parâmetro é usado para dividir automaticamente os dados em conjuntos de treinamento e validação 
    validation_split=0.3
    )


In [11]:

BATCH_SIZE = 50
TARGET_SIZE=(100, 120) # é ao contrário aqui essa bussanha
CMODE='grayscale'
CLASS_MODE='categorical'
SHUFFLE = True
# Imagens para treinamento
train_generator = train_datagen.flow_from_directory(
    xray_directory,
    target_size=TARGET_SIZE,
    batch_size=BATCH_SIZE,
    color_mode=CMODE, 
    class_mode=CLASS_MODE,
    shuffle=SHUFFLE,
    subset='training'
)


    
print(train_generator.class_indices)

# Imagens para validação
validation_generator = train_datagen.flow_from_directory(
        xray_directory,
        target_size=TARGET_SIZE,
        batch_size=BATCH_SIZE,
        color_mode=CMODE,
        class_mode=CLASS_MODE,
        shuffle=SHUFFLE,
        subset = 'validation'
        )

Found 2658452 images belonging to 53 classes.
{'.ipynb_checkpoints': 0, 'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7, 'H': 8, 'I': 9, 'J': 10, 'K': 11, 'L': 12, 'M': 13, 'N': 14, 'O': 15, 'P': 16, 'Q': 17, 'R': 18, 'S': 19, 'T': 20, 'U': 21, 'V': 22, 'W': 23, 'X': 24, 'Y': 25, 'Z': 26, 'a': 27, 'b': 28, 'c': 29, 'd': 30, 'e': 31, 'f': 32, 'g': 33, 'h': 34, 'i': 35, 'j': 36, 'k': 37, 'l': 38, 'm': 39, 'n': 40, 'o': 41, 'p': 42, 'q': 43, 'r': 44, 's': 45, 't': 46, 'u': 47, 'v': 48, 'w': 49, 'x': 50, 'y': 51, 'z': 52}
Found 1139320 images belonging to 53 classes.


In [12]:
# Testar o gerador de dados
for data_batch, labels_batch in train_generator:
    print("Imagens: ", data_batch.shape)
    print("Labels: ", labels_batch.shape)
    break  # Apenas testar um batch


Imagens:  (50, 100, 120, 1)
Labels:  (50, 53)


# treinamento do modelo

In [13]:
history = model.fit(train_generator, epochs = 25, validation_data=validation_generator, callbacks=callbacks_list)


Epoch 1/25


/home/picg/anaconda3/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1728858960.201843    6809 service.cc:145] XLA service 0x76d56c003710 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1728858960.201871    6809 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 4070, Compute Capability 8.9
2024-10-13 19:36:00.228883: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-10-13 19:36:00.328013: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
I0000 00:00:1728

   11/53170 ━━━━━━━━━━━━━━━━━━━━ 16:12 18ms/step - accuracy: 0.0130 - loss: 3.9864 

I0000 00:00:1728858962.714437    6809 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


 3948/53170 ━━━━━━━━━━━━━━━━━━━━ 15:43 19ms/step - accuracy: 0.1605 - loss: 3.2488

KeyboardInterrupt: 

In [ ]:
aux = history
history = history.history.keys() #paramestros de avaliação accuracy, loss
# Exemplo de grafico para ver taxa de acerto por taxa de erro 
plt.plot(aux.history['accuracy'])
plt.plot(aux.history['loss'])
plt.title('Erro e taxa de acerto durante o treinamento')
plt.xlabel('Época')
plt.ylabel('Taxa de acerto e erro')
plt.legend(['Taxa de acerto', 'Erro']);

In [ ]:
hist_df = pd.DataFrame(aux.history)
print(hist_df)

In [ ]:
# Salvar o DataFrame em um arquivo CSV
hist_df.to_csv('training_history_cropped_original.csv', index=False)

# Carregar histórico de treinamento anterior

In [ ]:
# Carregar os dados do CSV
loaded_history_df = pd.read_csv('training_history.csv')
# Exibir os dados carregados (opcional)
print(loaded_history_df)

In [ ]:
# Plotar a perda de treinamento e validação
plt.plot(loaded_history_df['loss'], label='Loss (train)')
plt.plot(loaded_history_df['val_loss'], label='Loss (validation)')
plt.title('Loss durante o treinamento')
plt.xlabel('Épocas')
plt.ylabel('Perda')
plt.legend()
plt.show()

# Plotar a precisão de treinamento e validação
plt.plot(loaded_history_df['accuracy'], label='Accuracy (train)')
plt.plot(loaded_history_df['val_accuracy'], label='Accuracy (validation)')
plt.title('Precisão durante o treinamento')
plt.xlabel('Épocas')
plt.ylabel('Precisão')
plt.legend()
plt.show()

In [ ]:
#Comparação de LOSS e ACCURACY entre as epocas de treino da rede

# Plot Loss
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(loaded_history_df['loss'], label='train loss')
plt.plot(loaded_history_df['val_loss'], label='val loss')
plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plot Accuracy
plt.subplot(1, 2, 2)
plt.plot(loaded_history_df['accuracy'], label='train accuracy')
plt.plot(loaded_history_df['val_accuracy'], label='val accuracy')
plt.title('Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()


In [ ]:
#Comparação de Loss e Accuracy de redes distintas

import numpy as np

def plot_comparison(results, metric='accuracy'):
    models = list(results.keys())
    final_accuracy = [results[model][metric][-1] for model in models]
    
    plt.figure(figsize=(10, 5))
    plt.bar(models, final_accuracy)
    plt.title(f'Comparação de {metric} entre diferentes redes')
    plt.ylabel(metric.capitalize())
    plt.xlabel('Modelos')
    plt.show()


In [ ]:
#Grafico de curvas para comparação entre redes
def plot_comparison_curves(results, metric='accuracy'):
    plt.figure(figsize=(10, 5))
    
    for model, history in results.items():
        plt.plot(history[metric], label=f'{model} - train')
        plt.plot(history['val_' + metric], '--', label=f'{model} - val')
    
    plt.title(f'Comparação de {metric.capitalize()} entre Redes')
    plt.xlabel('Épocas')
    plt.ylabel(metric.capitalize())
    plt.legend()
    plt.show()


In [ ]:
#Matriz Confusão
from sklearn.metrics import confusion_matrix
import seaborn as sns

def plot_confusion_matrix(y_true, y_pred, model_name):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 7))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f'Matriz de Confusão - {model_name}')
    plt.ylabel('Verdadeiro')
    plt.xlabel('Predito')
    plt.show()



In [ ]:
#Boxplot
def plot_boxplot(results, metric='accuracy'):
    data = [history[metric] for history in results.values()]
    labels = results.keys()

    plt.figure(figsize=(10, 6))
    plt.boxplot(data, labels=labels)
    plt.title(f'Boxplot de {metric.capitalize()} entre Redes')
    plt.ylabel(metric.capitalize())
    plt.show()


In [ ]:
# defini caminho para pasta de teste
test_directory = 'dataset/test'
# lista contendo os nomes dos arquivos e diretórios presentes no caminho fornecido
os.listdir(test_directory)

In [ ]:
# Mudar a escala das imagens
test_gen = ImageDataGenerator(rescale=1./255)

test_generator = test_gen.flow_from_directory(batch_size = 13, directory = test_directory,
                                              shuffle = True, target_size = (256, 256),
                                              class_mode = 'categorical')

evaluate = model.evaluate(test_generator)

len(os.listdir(test_directory))

In [ ]:
# aplica a rede treinada em um conjunto de teste de imagens e armazena os resultados
prediction = []
original = []
image = []

# percorre as subpastas no diretório test_directory, assumindo que cada subpasta contém imagens de uma determinada classe
for i in range(len(os.listdir(test_directory))):
  # percorre os arquivos dentro de cada subpasta e processa cada imagem
  for item in os.listdir(os.path.join(test_directory, str(i))):
    
    #Etapas de processamento de img

    img = cv2.imread(os.path.join(test_directory, str(i), item))
    img = cv2.resize(img, (256, 256))
    image.append(img)
    img = img / 255
    img = img.reshape(-1, 256, 256, 3)

    #rede neural aplicada
    predict = model.predict(img)
    predict = np.argmax(predict)
    prediction.append(predict)
    original.append(i)

In [ ]:
accuracy_score(original, prediction)

In [ ]:
fig, axes = plt.subplots(5, 5, figsize=(12,12))
axes = axes.ravel()
for i in np.arange(0, 25):
  axes[i].imshow(image[i])
  axes[i].set_title('Previsão={}\nTrue={}'.format(str(labels_names[prediction[i]]), str(labels_names[original[i]])))
  axes[i].axis('off')
plt.subplots_adjust(wspace = 1.2)

In [ ]:
labels_names = {0: 'SoldasCorretas', 1: 'SoldasErradas'}

# Carrgar Keras e teste

In [1]:
from keras.models import load_model
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# Carregar o modelo previamente salvo
model_load = load_model('weights_cropped_original.keras')

# Carregar a imagem e convertê-la para um array numpy em escala de cinza
image = np.array(Image.open("/home/picg/TCC/GeradorBase/font_results/cropped/a/a-tat-AlexBrush-Regular.png").convert('L'))
#image = np.array(Image.open("/home/picg/TCC/GeradorBase/white.png").convert('L'))
# Redimensionar a imagem para o tamanho esperado pela CNN (supondo 120x100)
temp_resized = cv2.resize(image, (120, 100))

print(temp_resized.shape)

# Normalizar os valores dos pixels para [0, 1]
normalized = temp_resized / 255.0

# Adicionar uma dimensão para corresponder ao formato de entrada esperado pelo modelo
final = normalized.reshape(1, 120, 100, 1)  # '1' aqui é o batch size

# Fazer a predição com o modelo carregado
prediction = model_load.predict(final)

# Exibir o resultado da predição
print("Predição:", prediction)


2024-10-29 17:44:21.364800: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-29 17:44:21.720711: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-10-29 17:44:22.215858: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-29 17:44:22.235865: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA no

(100, 120)


I0000 00:00:1730234662.937333   99990 service.cc:145] XLA service 0x7b29e4005920 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1730234662.937358   99990 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 4070, Compute Capability 8.9
2024-10-29 17:44:22.942351: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-10-29 17:44:22.983513: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Predição: [[2.65324260e-21 2.22141061e-08 4.02441991e-08 7.96353220e-07
  4.69154278e-17 3.33364904e-02 6.07140436e-08 2.21670163e-03
  2.99124657e-13 6.74922873e-09 1.73026848e-07 1.02861079e-11
  6.69583000e-11 1.37064017e-31 6.57282320e-15 2.60236299e-12
  2.30744357e-09 6.57312967e-06 6.10812867e-09 1.19912951e-08
  6.28709245e-07 3.54019952e-10 7.66138541e-12 1.02770216e-16
  2.14316217e-12 2.19572428e-07 3.03312001e-04 7.50980256e-09
  7.91043421e-05 1.02811624e-04 1.08639642e-11 8.07612948e-03
  1.40869111e-06 4.74894932e-08 3.89248999e-06 6.07829643e-05
  3.73903231e-06 7.10019290e-01 5.65024675e-06 1.64341121e-15
  6.94630298e-10 3.00285841e-08 2.19215690e-09 5.13844398e-08
  1.95863468e-05 3.59848173e-06 2.21641903e-06 9.63321867e-09
  4.28584053e-06 1.24840971e-10 1.48481263e-06 5.19901455e-08
  2.45750800e-01]]


I0000 00:00:1730234664.209581   99990 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


In [ ]:
plt.imshow(temp_resized, cmap="gray")

In [ ]:
#image = np.array(Image.open("/home/picg/TCC/GeradorBase/font_results/normal/a/a-mam-GrandHotel-Regular.png"))
plt.imshow(image, cmap="gray")

In [ ]:
import numpy as np

# Achar a classe com maior probabilidade
predicted_class = np.argmax(prediction)
'''
{'.ipynb_checkpoints': 0, 'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7, 'H': 8, 'I': 9, 'J': 10, 'K': 11, 'L': 12, 'M': 13, 'N': 14, 'O': 15, 'P': 16, 'Q': 17, 'R': 18, 'S': 19, 'T': 20, 'U': 21, 'V': 22, 'W': 23, 'X': 24, 'Y': 25, 'Z': 26, 'a': 27, 'b': 28, 'c': 29, 'd': 30, 'e': 31, 'f': 32, 'g': 33, 'h': 34, 'i': 35, 'j': 36, 'k': 37, 'l': 38, 'm': 39, 'n': 40, 'o': 41, 'p': 42, 'q': 43, 'r': 44, 's': 45, 't': 46, 'u': 47, 'v': 48, 'w': 49, 'x': 50, 'y': 51, 'z': 52}
'''

# Definir labels (saidas) 
labels_names1 = {0: '.ipynb_checkpoints', 
                1: 'a',
                2: 'A',
                3: 'b',
                4: 'B',
                5: 'c',
                6: 'C',
                7: 'd',
                8: 'D',
                9: 'e',
                10: 'E', 
                11: 'f',
                12: 'F',
                13: 'g',
                14: 'G',
                15: 'h',
                16: 'H',
                17: 'i',
                18: 'I',
                19: 'j',
                20: 'J', 
                21: 'k',
                22: 'K',
                23: 'l',
                24: 'L',
                25: 'm',
                26: 'M',
                27: 'n',
                28: 'N',
                29: 'o',
                30: 'O', 
                31: 'p',
                32: 'P',
                33: 'q',
                34: 'Q',
                35: 'r',
                36: 'R',
                37: 's',
                38: 'S',
                39: 't',
                40: 'T', 
                41: 'u',
                42: 'U',
                43: 'v',
                44: 'V',
                45: 'w',
                46: 'W',
                47: 'x',
                48: 'X',
                49: 'y',
                50: 'Y', 
                51: 'z',
                52: 'Z',
                }
labels_names = {
    0: "g",
    1: "V",
    2: "W",
    3: "r",
    4: "i",
    5: "a",
    6: "G",
    7: "I",
    8: "s",
    9: "J",
    10: "y",
    11: "z",
    12: "t",
    13: "c",
    14: "Q",
    15: "u",
    16: "E",
    17: "o",
    18: "H",
    19: "B",
    20: "U",
    21: "N",
    22: "F",
    23: "m",
    24: "n",
    25: "v",
    26: "T",
    27: "D",
    28: "A",
    29: "e",
    30: "w",
    31: "O",
    32: ".ipynb_checkpoints",
    33: "S",
    34: "K",
    35: "l",
    36: "C",
    37: "q",
    38: "p",
    39: "Z",
    40: "j",
    41: "h",
    42: "f",
    43: "L",
    44: "M",
    45: "b",
    46: "d",
    47: "P",
    48: "x",
    49: "X",
    50: "Y",
    51: "R",
    52: "k"
}
print(f"A classe prevista é - {predicted_class}: {labels_names[predicted_class]}")


In [ ]:
model_load.summary()